# Práctica 2. Resolución de problemas con búsqueda 
## Ingeniería del Conocimiento    2025/2026
### Prof. Juan A. Recio García

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en representar estados y acciones, la comprobación de objetivo y el coste de las acciones.
La siguiente clase Problem (de la librería AIMA) representa este esquema general de cualquier problema de espacio de estados. Un problema concreto será una subclase de Problem, y requerirá implementar
* estado_inicial, es_estado_final(_), acciones(_), aplica(_,_), coste_de_aplicar_accion y eventualmente __init__ 



La librería aima se puede instalar directamente con pip, pero te instalará muchísimas dependencias. Por ahora no hace falta, ha que para esta práctica se usarán los archivos _search.py y utils.py_ de AIMA que se incluyen junto al notebook

In [1]:
from search import *

### Copiamos aquí la definición de la clase Problem de la librería AIMA para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver algunos ejemplos de cómo __definir un problema como subclase
de Problem__. En concreto, el problema de los misioneros, visto en clase.

# EJERCICIO 1: Problema de los misioneros

Para representar el problema simplemente utilizamos una tupla (M, C, B) donde:
* M es el número de misioneros en el lado inicial (donde se encuentra el bote B)
* C es el número de caníbales en el lado inicial
* B es la posición del bote (0: lado inicial, 1: lado final)

Representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la cantidad de misioneros y canibales que se mueven en la canoa. Por ejemplo:
* ('1c', (0, 1)): mueve 1 canibal al otro lado
* ('1m1c', (1, 1)): mueve 1 canibal y 1 misionero al otro lado

In [2]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
# Aunque no es necesario puedes hacer tu propia implementación del problema 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema. En este caso el estado inicial y objetivo se pasan como parámetros'''
        Problem.__init__(self, initial, goal)
        # En este caso representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido. La comprobación de las precondiciones se realiza en una función auxiliar is_valid
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3 
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
        determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:  #si la canoa está en el lado 0
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [3]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

In [4]:
# Comprobamos el estado inicial de nuestro problema
misioneros.initial

(3, 3, 0)

In [5]:
# Acciones aplicables en el estado inicial
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

In [6]:
# Resultado de aplicar una acción a un estado
misioneros.result(misioneros.initial, ('2c', (0, 2)))

(3, 1, 1)

In [7]:
# Comprobamos si un estado dado es objetivo
misioneros.goal_test(misioneros.initial)

False

In [8]:
# Comprobamos que el estado final identificado como el objetivo
misioneros.goal_test((0, 0, 1))

True

## Búsquedas

Por ejemplo, para resolver el problema de los misioneros con 
el método de busqueda en anchura la llamada sería:  


In [9]:
# el método solution() de un nodo devuelve la secuencia de acciones que llevan desde el estado inicial al estado final
breadth_first_tree_search(misioneros).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

La búsqueda en profundidad es:

In [10]:
# por profundidad
depth_first_tree_search(misioneros).solution()

KeyboardInterrupt: 

### EJERCICIO 1: ¿Por qué no funciona?

Explícalo. Piensa en alguna solución

EXPLICALO AQUÍ


El siguiente método realiza búsqueda en profundidad con control de repetición de estados

In [11]:
depth_first_graph_search(misioneros).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

In [12]:
# Método de búsqueda por costo uniforme
uniform_cost_search(misioneros).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [13]:
# Búsqueda por profundidad limitada
depth_limited_search(misioneros, 10)

'cutoff'

### EJERCICIO: ¿Por qué no funciona depth_limited_search?

EXPLICALO AQUÍ

# EJERCICIO 2: 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo: 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales

Completa la implementación parcial de la clase

In [ ]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. 
    
    Las cuatro acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", 
    "Mover hueco abajo", 
    "Mover hueco izquierda" y
    "Mover hueco derecha"."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
        self.actions = ['Mover hueco arriba', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco izquierda']

    def actions(self, estado):
        pos_hueco = estado.index(0)  # busco la posicion del 0
        accs = list()

        if(pos_hueco):
            pass
        
        return accs

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        def swap(x):
            l[pos_hueco], l[pos_hueco+x] = l[pos_hueco+x], l[pos_hueco]
        
        if accion == 'Mover hueco arriba':
            if pos_hueco < 3:
                swap(pos_hueco - 3)
        elif accion == 'Mover hueco abajo':
            if pos_hueco > 5:
                swap(pos_hueco + 3)
        elif accion == 'Mover hueco derecha':
            if pos_hueco in [2, 5, 8]:
                swap(pos_hueco + 1)
        elif accion == 'Mover hueco izquierda':
            if pos_hueco in [0, 3, 6]:
                swap(pos_hueco - 1)

        return tuple(l)

    def h(self, node):
        return sum(abs(n - self.goal.index(n)) for n in node)
    
    def coste_de_aplicar_accion(self, s, a):
        return 1

####  Prueba los siguientes ejemplos.

In [ ]:
p8 = Ocho_Puzzle((2, 3, 8, 1, 6, 4, 7, 0, 5))
p8.initial

In [ ]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [ ]:
p8.result(p8.initial,"Mover hueco arriba")

In [ ]:
p8.result(p8.initial,"Mover hueco abajo")

### EJERCICIO: ¿Por qué da error?

EXPLÍCALO AQUÍ

## Prueba los distintos métodos de búsqueda que hemos visto antes e indica a continuación qué está ocurriendo

In [ ]:
#Ejecúta los métodos aquí

Explica aquí qué está ocurriendo

## Ahora prueba los métodos de búsqueda con el siguiente estado inicial e indica a continuación qué está ocurriendo

In [ ]:
p8 = Ocho_Puzzle((2, 3, 8, 1, 6, 4, 7, 0, 5))

In [ ]:
#Ejecúta los métodos aquí

Explica aquí qué está ocurriendo

# EJERCICIO 3: Puzzle del granjero, cabra, lobo y la col

Implementa el problema a partir de la descripción proporcionada en la hoja de ejercicios:

Un granjero está en la orilla de un río con una cabra, un lobo y una col . Su
objetivo es llevarlos a la otra orilla, teniendo en cuenta las siguientes restricciones:

* El granjero tiene una barca en la que sólo puede llevar una “cosa” a la vez y la
barca tiene que ser conducida por el granjero.
* En cualquiera de las orillas o la barca, el lobo se come a la cabra si no está el
granjero.
* En cualquiera de las orillas o la barca, la cabra se come a la col si no está el
granjero.

In [ ]:
class GranjeroLoboOvejaCol(Problem):
    """
    Problema del granjero que debe cruzar un río con un lobo, una cabra y una col.
    
    DESCRIPCIÓN:
    - El granjero tiene una barca que solo puede llevar UNA cosa a la vez
    - La barca debe ser conducida por el granjero
    - Si el granjero no está presente:
        * El lobo se come a la cabra
        * La cabra se come la col
    
    REPRESENTACIÓN DEL ESTADO:
    Usamos una tupla de 4 elementos booleanos: (granjero, lobo, cabra, col)
    - False = en la orilla izquierda (origen)
    - True = en la orilla derecha (destino)
    
    Ejemplo: (False, False, True, False) significa:
    - Granjero en orilla izquierda
    - Lobo en orilla izquierda  
    - Cabra en orilla derecha
    - Col en orilla izquierda
    
    ESTADO INICIAL: (False, False, False, False) - todos en orilla izquierda
    ESTADO OBJETIVO: (True, True, True, True) - todos en orilla derecha
    
    ACCIONES:
    - 'solo': El granjero cruza solo
    - 'lobo': El granjero cruza con el lobo
    - 'cabra': El granjero cruza con la cabra
    - 'col': El granjero cruza con la col
    """
    
    def __init__(self):
        """
        Inicializa el problema con estado inicial y objetivo.
        """

    
    def _is_valid(self, state):
        """
        Verifica si un estado es válido (no hay conflictos).
        
        Un estado es inválido si:
        - El lobo está con la cabra sin el granjero (lobo se come cabra)
        - La cabra está con la col sin el granjero (cabra se come col)
        
        Args:
            state: tupla (granjero, lobo, cabra, col)
            
        Returns:
            bool: True si el estado es válido, False en caso contrario
        """

    
    def actions(self, state):
        """
        Devuelve las acciones posibles desde un estado dado.
        
        Las acciones dependen de:
        1. Dónde está el granjero (determina qué puede llevar)
        2. Qué elementos están en la misma orilla que el granjero
        3. Si el estado resultante es válido
        
        Args:
            state: tupla (granjero, lobo, cabra, col)
            
        Returns:
            list: Lista de acciones válidas ('solo', 'lobo', 'cabra', 'col')
        """
        
    
    def result(self, state, action):
        """
        Devuelve el estado resultante de aplicar una acción.
        
        Args:
            state: tupla (granjero, lobo, cabra, col)
            action: string con la acción ('solo', 'lobo', 'cabra', 'col')
            
        Returns:
            tuple: Nuevo estado después de aplicar la acción
        """



### Ejecuta los distintos métodos de búsqueda, identifíca los problemas encontrados

In [ ]:
problema_granjero = GranjeroLoboOvejaCol()

In [ ]:
breadth_first_tree_search(problema_granjero).solution()

In [ ]:
depth_first_tree_search(problema_granjero).solution()

In [ ]:
breadth_first_graph_search(problema_granjero).solution()

In [ ]:
depth_first_graph_search(problema_granjero).solution()

In [ ]:
uniform_cost_search(problema_granjero).solution()

INDICA AQUÍ LOS PROBLEMAS ENCONTRADOS   

# EJERCICIO 4: Problema de las jarras de agua

## Formaliza el siguiente problema con AIMA y prueba los métodos de búsqueda ciegos

Se dispone de dos jarras sin marcas de medición:

* Una jarra con capacidad de 4 litros
* Una jarra con capacidad de 3 litros

Ambas jarras están inicialmente vacías. Se tiene acceso a una fuente ilimitada de agua (un grifo) y un sumidero donde verter agua.

*Objetivo*: Conseguir exactamente 2 litros de agua en alguna de las jarras.

Operaciones permitidas

* Llenar completamente una jarra desde el grifo
* Vaciar completamente una jarra en el sumidero
* Verter agua de una jarra a otra, hasta que:

    - La jarra origen se vacíe, o
    - La jarra destino se llene (lo que ocurra primero)



Formalización como problema de búsqueda

* Estado: Tupla (j4, j3) donde j4 ∈ [0,4] y j3 ∈ [0,3] representan los litros en cada jarra
* Estado inicial: (0, 0) — ambas jarras vacías
* Estado objetivo: Cualquier estado donde j4 = 2 o j3 = 2
* Acciones: llenar_4, llenar_3, vaciar_4, vaciar_3, verter_4_a_3, verter_3_a_4
* Coste1 por cada acción (uniforme)

Restricciones

* No se puede medir el agua de ninguna otra forma (no hay marcas)
* No se puede llenar parcialmente una jarra desde el grifo
* No se puede estimar "a ojo" una cantidad